In [ ]:
#!apt-get remove python3-opencv
#!apt-get install python3-opencv
#!pip uninstall opencv_python -y
#!pip install -U opencv_python --user
#!rm -rf test/

In [ ]:
!pip install boto3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.5/135.5 KB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6 KB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.6/10.6 MB 52.0 MB/s eta 0:00:00


In [ ]:
import os
import cv2
import boto3
import numpy as np
from google.colab.patches import cv2_imshow
from IPython.display import clear_output 

In [ ]:
#boto client
aws_access_key_id = 'SECRET_KEY'
aws_secret_access_key = 'SECRET_ACCESS_KEY'
aws_session_token= 'SECRET_TOKEN'
bucket_name = 's3-data-raw-tcc'

s3 = boto3.client('s3', region_name="us-east-1", aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key, aws_session_token=aws_session_token)

In [ ]:
# Create a reusable Paginator
paginator = s3.get_paginator('list_objects')

# Create a PageIterator from the Paginator
page_iterator = paginator.paginate(Bucket=bucket_name)

In [ ]:
for page in page_iterator:
  for obj in page['Contents']:
    key = obj['Key']
    path, filename = os.path.split(key)
    if key[-4:] == '.mp4':
      if not os.path.exists(path):
        os.makedirs(path)
      print("Downloading...: "+key)
      s3.download_file(bucket_name, key, key)

In [ ]:
!du -h ./s3-data-raw-tcc

1.5G	./s3-data-raw-tcc/Anomaly-Videos-Part-4/Stealing
646M	./s3-data-raw-tcc/Anomaly-Videos-Part-4/Shoplifting
612M	./s3-data-raw-tcc/Anomaly-Videos-Part-4/Vandalism
2.7G	./s3-data-raw-tcc/Anomaly-Videos-Part-4
2.1G	./s3-data-raw-tcc/Anomaly-Videos-Part-3/Robbery
663M	./s3-data-raw-tcc/Anomaly-Videos-Part-3/Shooting
2.7G	./s3-data-raw-tcc/Anomaly-Videos-Part-3
1.5G	./s3-data-raw-tcc/Anomaly-Videos-Part-2/Burglary
1.5G	./s3-data-raw-tcc/Anomaly-Videos-Part-2
632M	./s3-data-raw-tcc/Anomaly-Videos-Part-1/Assault
632M	./s3-data-raw-tcc/Anomaly-Videos-Part-1
7.5G	./s3-data-raw-tcc


In [ ]:
!find . -size +50M -name "*.mp4" -exec rm -i {} \;

In [ ]:
def make_imgs(path_file, path_absolut):
  cap = cv2.VideoCapture(path_file)

  fps = cap.get(cv2.CAP_PROP_FPS)

  count=0
  # running the loop 
  while (cap.isOpened()): 
      # extracting the frames 
      ret, img = cap.read() 
      if ret == True:
          count += 1
          # converting to gray-scale 
          gray8_image = np.zeros((120, 160), dtype=np.uint8)
          gray8_image = cv2.normalize(img, gray8_image, 0, 255, cv2.NORM_MINMAX)
          gray8_image = np.uint8(gray8_image)

          inferno_palette = cv2.applyColorMap(gray8_image, cv2.COLORMAP_INFERNO)
          
          resized_img = cv2.resize(inferno_palette, (224,224))

          if count % int(fps) == 0:
              frame_name = os.path.join(path_absolut, filename[:-4]+'_frame{:04d}.jpg'.format(count))
              cv2.imwrite(frame_name, resized_img)
              print("Saving: "+frame_name)
      else:
          break

  # closing the window
  cap.release()
  cv2.destroyAllWindows()

In [ ]:
# 52'
import os
import cv2

if not os.path.exists('./s3-data-preprocessed-tcc'):
    os.makedirs('s3-data-preprocessed-tcc')

for page in page_iterator:
  for path_name in page['Contents']:
    full_path_file = path_name['Key']

    path, filename = os.path.split(full_path_file)
    path_imgs = path.replace('s3-data-raw-tcc', 's3-data-preprocessed-tcc')
    print("Doing: /content/"+full_path_file)
    if not os.path.exists(path_imgs):
      os.makedirs(path_imgs)

    make_imgs('/content/'+full_path_file, '/content/'+path_imgs)

In [ ]:
file_list = []
for root, dirs, files in os.walk('/content/s3-data-preprocessed-tcc/Anomaly-Videos-Part-4/Vandalism/'):
    for file in files:
        if file[-4:] == ".jpg":
          file_path = os.path.join(root, file)
          file_list.append(file_path)

In [ ]:
len(file_list)

3334

In [ ]:
for file_path in file_list:
    # Determine o caminho no S3 onde o arquivo será armazenado
    s3_path = os.path.relpath(file_path, '/content/').replace('\\', '/')
    s3_path = f'{bucket_name}/{s3_path}'
    s3_path = s3_path.replace('s3-data-raw-tcc/','')
    # upload the file to the new directory in the bucket
    s3.upload_file(file_path, bucket_name, s3_path)

    print(f'O arquivo {s3_path} foi enviado com sucesso para o bucket {bucket_name}')

In [ ]:
text = '''
[default]
aws_access_key_id = SECRET_KEY
aws_secret_access_key = SECRET_ACCESS_KEY
region = us-east-1
aws_session_token = SECRET_TOKEN
'''
path = "/content/awscli.ini"
with open(path, 'w') as f:
   f.write(text)
!cat /content/awscli.ini

In [ ]:
import os
!export AWS_SHARED_CREDENTIALS_FILE=/content/awscli.ini
path = "/content/awscli.ini"
os.environ['AWS_SHARED_CREDENTIALS_FILE'] = path
print(os.environ['AWS_SHARED_CREDENTIALS_FILE'])

/content/awscli.ini


In [ ]:
!aws s3 cp /content/s3-data-preprocessed-tcc/Anomaly-Videos-Part-4/Vandalism/ s3://s3-data-raw-tcc/s3-data-preprocessed-tcc/Anomaly-Videos-Part-4/Vandalism/ --recursive